In [10]:
import sys
import os 
import nest_asyncio

# Sanity check
print(sys.executable)
nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "sk-"
#os.environ[
#    "AZURE_OPENAI_ENDPOINT"
#] = "https://<your-resource-name>.openai.azure.com/"
#os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

/home/amor/Documents/code_dw/human_in_the_loop_workflow_demo/venv/bin/python3.11


In [18]:
import os
from pydantic import BaseModel, Field
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine


In [12]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O './paul_graham_essay.txt'

--2024-12-01 15:58:57--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘./paul_graham_essay.txt’

./paul_graham_essay 100%[===================>]  73,28K  --.-KB/s    in 0,02s   

2024-12-01 15:58:57 (3,71 MB/s) - ‘./paul_graham_essay.txt’ saved [75042/75042]



## 1 - RAG 

Using the BM25 retriever system 

In [13]:

documents = SimpleDirectoryReader(
    input_files=["./paul_graham_essay.txt"],
).load_data()
splitter = SentenceSplitter(chunk_size=256)
nodes = splitter.get_nodes_from_documents(documents)
retriever_top_5 = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=5,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [14]:
rez = retriever_top_5.retrieve("computer")

print(rez[0])

Node ID: 9f5abd83-5775-4820-a5d5-e64d47be56e9
Text: So I'm not surprised I can't remember any programs I wrote,
because they can't have done much. My clearest memory is of the moment
I learned it was possible for programs not to terminate, when one of
mine didn't. On a machine without time-sharing, this was a social as
well as a technical error, as the data center manager's expression
made clear....
Score:  1.289



In [15]:
print(rez[1].text)

It was meant to be a formal model of computation, an alternative to the Turing machine. If you want to write an interpreter for a language in itself, what's the minimum set of predefined operators you need? The Lisp that John McCarthy invented, or more accurately discovered, is an answer to that question. [19]

McCarthy didn't realize this Lisp could even be used to program computers till his grad student Steve Russell suggested it. Russell translated McCarthy's interpreter into IBM 704 machine language, and from that point Lisp started also to be a programming language in the ordinary sense. But its origins as a model of computation gave it a power and elegance that other languages couldn't match. It was this that attracted me in college, though I didn't understand why at the time.

McCarthy's 1960 Lisp did nothing more than interpret Lisp expressions. It was missing a lot of things you'd want in a programming language. So these had to be added, and when they were, they weren't define

In [19]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever_top_5,
    response_synthesizer=response_synthesizer,
)
response = query_engine.query("Who is Paul Graham.")

In [21]:
response.response

'Paul Graham is a hardworking individual who dedicated a significant amount of time and effort to working on a project called Bel. He faced challenges while working on this project, such as wrestling with bugs and dealing with complex problems. Despite these difficulties, he found the work satisfying and rewarding. Additionally, he is known for his involvement in creating the Summer Founders Program, which attracted a large number of applicants, including notable individuals like Justin Kan, Emmett Shear, Aaron Swartz, and Sam Altman.'

## 2 - Exercise : 

Combine it with Workflows

Create a workflow that : 
- Search for the best quote about the user query
- Make a rap about it




In [8]:

class ContextualGrahamRapWorkflow(Workflow):
    
    @step
    def do(self, ev: StartEvent) -> StopEvent:
        return StopEvent()



In [9]:
w = ContextualGrahamRapWorkflow()

r = await w.run()